# The other files
# System.py

In [ ]:
import numpy as np


class MotorSystem():
    def __init__(self,f_c = 0.6,b=0.4,E_max = 5.0, noise_std = 0.02):

        #Initializing variables

        self.f_c = f_c
        self.b = b
        self.E_max = E_max
        self.noise_std = noise_std

        # Initializing System variables
        self.x = 0.0
        self.v = 0.0

    # Reseting the System
    def reset(self,x,v):
        self.x = x
        self.v = v

    # To get the Saturated Value of E incase the Throttle is too much
    def saturation_E (self,E):
        return np.clip(E,-self.E_max,self.E_max)

    def step(self,E,dt):
        self.E = self.saturation_E(E)
        self.noise = np.random.randn()*self.noise_std
        self.x_dot = self.v
        self.v_dot = self.E - self.b*self.v -self.f_c*np.tanh(self.v)+self.noise
        self.x+=self.x_dot*dt
        self.v+=self.v_dot*dt
        return self.x , self.v

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Question 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from System import MotorSystem
from PID import PID


# Simulation Parameters (Do not change)
dt = 0.01
T = 10.0
steps = int(T/dt)

position_choices = [-4, -2, 0, 2, 5] #desired Positions, if you are stuck check line 57

Switching_interval = 2.5 # Switching Time
switching_steps = int(Switching_interval/dt)


# Initialize system and PID controller
motor = MotorSystem()
pid = PID(kp=6.0, ki=0.8, kd=1.0)
pid.reset()

# Data Storage
time = np.zeros(steps)
x_log = np.zeros(steps)
v_log = np.zeros(steps)
E_log = np.zeros(steps)
x_des_log = np.zeros(steps)

# Initial desired position
x_des = np.random.choice(position_choices) # For selecting Intial position

# Ensure system starts from zero state
motor.reset(0.0, 0.0)

for i in range(steps):

    time[i] = i*dt

    # Fixed desired setpoint (chosen once at start, no switching)
    # x_des is set before the loop and remains constant

    # Compute control action (PID on position)
    error = x_des - motor.x
    E = pid.update(dt, error)

    # Apply control and step the system
    x, v = motor.step(E, dt)

    # Logs
    x_log[i] = x
    v_log[i] = v
    E_log[i] = E
    x_des_log[i] = x_des

# Plotting results
fig, axs = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Position
axs[0].plot(time, x_log, label="Actual Position", lw=2)
axs[0].plot(time, x_des_log, "--", label="Desired Position", lw=2)
axs[0].set_ylim(min(position_choices) - 1, max(position_choices) + 1)
axs[0].set_ylabel("Position")
axs[0].set_title("Direct Position PID")
axs[0].legend()
axs[0].grid()

# Effort
axs[1].plot(time, E_log, label="Effort (E)")
axs[1].set_ylabel("Effort")
axs[1].set_xlabel("Time (s)")
axs[1].legend()
axs[1].grid()

plt.tight_layout()
plt.show()




On tuning with different p,i,d terms I have observed changes in the graph. First as I started with only kp as 3, the graph was fluctuating so much which is not suitable for a smooth motion. So then I have put ki to 0.4. Now the fluctuation was little but the position is still not smooth. Then I put kd to 1, which changed the graph completely. Of course there is only one bump and then it reached the desired value smoothly. So my def of good kp:3-7, ki: 0-0.8, kd- 0.5-2 . The above code also has a plot of error tracking.

# Question 2

In [ ]:
#Velocity control
import numpy as np
import matplotlib.pyplot as plt
from System import MotorSystem
from PID import PID


# Simulation Parameters (Do not change)
dt = 0.01
T = 10.0
steps = int(T/dt)

velocity_choices = [-1, -5, 3, 2, 6]
Switching_interval = 2.5
switching_steps = int(Switching_interval/dt)

"""
code for initializing System and PID controller
"""
# Initialize system and PID controller
motor = MotorSystem()
pid = PID(kp=4.0, ki=0.5, kd=0.1)
pid.reset()

# Data Storage
time = np.zeros(steps)
v_log = np.zeros(steps)
v_des_log = np.zeros(steps)
E_log = np.zeros(steps)

# Single desired velocity (fixed)
v_des = np.random.choice(velocity_choices) # Selected once and not switched

# Ensure system starts from zero state
motor.reset(0.0, 0.0)

for i in range(steps):

    time[i] = i*dt

    # Compute control action (PID on velocity)
    error = v_des - motor.v
    E = pid.update(dt, error)

    # Apply control and step the system
    x, v = motor.step(E, dt)

    # Logs
    v_log[i] = v
    E_log[i] = E
    v_des_log[i] = v_des

# Plotting results
fig, axs = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Velocity
axs[0].plot(time, v_log, label="Actual Velocity", lw=2)
axs[0].plot(time, v_des_log, "--", label="Desired Velocity", lw=2)
axs[0].set_ylim(min(velocity_choices) - 2, max(velocity_choices) + 2)
axs[0].set_ylabel("Velocity")
axs[0].set_title("Direct Velocity PID")
axs[0].legend()
axs[0].grid()

# Effort
axs[1].plot(time, E_log, label="Effort (E)")
axs[1].set_ylabel("Effort")
axs[1].set_xlabel("Time (s)")
axs[1].legend()
axs[1].grid()

plt.tight_layout()
plt.show()




# Question 3
# Cascaded Controller

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from System import MotorSystem
from PID import PID


"""
In this loop we are doing velocity control in the inner loop and
position control in outer loop.
"""

# Simulation Parameters (Do not Change)

dt = 0.01
T = 10.0
steps = int(T/dt)

x_desired_choices = [-4,-2,4,5,1,3,6]

switching_interval = 2.5
switching_steps = int(switching_interval/dt)

"""
Initialise System and PID

Create two PID controllers:
 - Outer PID (position -> velocity reference)
 - Inner PID (velocity -> effort)
"""
# Initialize system and PIDs
motor = MotorSystem()
pid_pos = PID(kp=2.0, ki=0.2, kd=0.5)   # outer loop (position)
pid_vel = PID(kp=6.0, ki=0.8, kd=0.3)   # inner loop (velocity)
pid_pos.reset()
pid_vel.reset()

# For logging purposes

time = np.zeros(steps)
x_log = np.zeros(steps)
x_des_log = np.zeros(steps)
v_log = np.zeros(steps)
v_ref_log = np.zeros(steps)
E_log = np.zeros(steps)

# Initial desired position
x_des = np.random.choice(x_desired_choices)

# Ensure system starts from zero state
motor.reset(0.0, 0.0)

# Velocity reference saturation (to avoid unrealistic refs)
v_ref_max = 10.0

for i in range(steps):
    time[i] = i * dt

    # Fixed desired setpoint (chosen once before the loop)
    # x_des was selected above and remains constant

    # Outer loop: position PID -> velocity reference
    pos_error = x_des - motor.x
    v_ref = pid_pos.update(dt, pos_error)
    v_ref = np.clip(v_ref, -v_ref_max, v_ref_max)

    # Inner loop: velocity PID -> effort
    vel_error = v_ref - motor.v
    E = pid_vel.update(dt, vel_error)

    # Apply control and step the system
    x, v = motor.step(E, dt)

    # Data Logging
    x_log[i] = x
    x_des_log[i] = x_des
    v_log[i] = v
    v_ref_log[i] = v_ref
    E_log[i] = E

# Plotting Part

fig, axs = plt.subplots(3, 1, figsize=(10, 9), sharex=True)

# Position
axs[0].plot(time, x_log, label="Actual Position")
axs[0].plot(time, x_des_log, "--", label="Desired Position")
axs[0].set_ylabel("Position")
axs[0].legend()
axs[0].grid()

# Velocity
axs[1].plot(time, v_log, label="Actual Velocity")
axs[1].plot(time, v_ref_log, "--", label="Velocity Reference")
axs[1].set_ylabel("Velocity")
axs[1].legend()
axs[1].grid()

# Effort
axs[2].plot(time, E_log, label="Effort")
axs[2].set_ylabel("Effort")
axs[2].set_xlabel("Time (s)")
axs[2].legend()
axs[2].grid()

plt.tight_layout()
plt.show()


Observations from the plot:
The outer loop (position) tracks the desired position slowly and smoothly.
The inner loop (velocity) responds much faster to velocity reference commands.
This separation allows each loop to be tuned independently for optimal performance:
Inner loop: Fast, high-gain, can quickly reject disturbances.
Outer loop: Slower, smooth, prevents overshoot in position.
**Result: The position settles closer to the setpoint with minimal oscillation, while the velocity quickly reaches its reference.
Without cascaded control, a single-loop position PID would have to compromise between fast response (which can overshoot) and stability (slower response). Cascading avoids this compromise.**

##  Effect of Aggressive Tuning of the Outer Loop

If the outer loop (position) is tuned **too aggressively** (high proportional gain or fast integral):

- It will produce **large velocity commands** to the inner loop.
- The inner loop might **saturate** or respond with **high overshoot**, causing:
  - Large spikes in control effort (bottom plot).  
  - Oscillations in both **position** and **velocity**.  
  - Potentially induces **instability**, especially if the inner loop cannot react fast enough to track the high-demand velocity reference.

💡 **Observation in my plot:**  
The outer loop is tuned **moderately** — position smoothly approaches **-4 units**.  

If it were more aggressive, we would see:

- Larger initial **spikes in velocity**.  
- **Overshoot** in position.  
- Effort plot showing **big peaks**.
